# Client

Demo of a client interacting with a remote agent that can use history.

See relevant documentation about agents:

* Creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
* Streaming with agents: https://python.langchain.com/docs/modules/agents/how_to/streaming#custom-streaming-with-events
* General streaming documentation: https://python.langchain.com/docs/expression_language/streaming

You can interact with this via API directly

In [2]:
import requests

inputs = {"input": {"input": "单词 shinnyy 的长度是多少？", "chat_history": []}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': {'output': 'The length of the word "shinnyy" is 7.'},
 'metadata': {'run_id': 'de9ca5b6-1930-4cfe-8d23-36fb1965cf1f',
  'feedback_tokens': []}}

You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [3]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

In [5]:
chat_history = []

while True:
    human = input("人类（输入Q/q退出）: ")
    if human in {"q", "Q"}:
        print('AI: 再见，人类')
        break
    ai = await remote_runnable.ainvoke({"input": human, "chat_history": chat_history})
    print(f"AI: {ai['output']}")
    chat_history.extend([HumanMessage(content=human), AIMessage(content=ai['output'])])

AI: Hi, I'm an AI assistant. I can help you with a variety of tasks, from answering questions to providing information and assistance. How can I assist you today?
AI: Yes, I can understand and respond in Chinese. How can I assist you in Chinese today?
AI: 你好，我是一个人工智能助手。我的主要任务是帮助用户解决问题、提供信息和帮助。今天我如何可以帮助你？
AI: 再见，人类


## Stream

Please note that streaming alternates between actions and observations. It does not stream individual tokens!

To stream individual tokens, we need to use the astream events endpoint (see below).

In [6]:
chat_history = []

while True:
    human = input("人类（输入Q/q退出）: ")
    if human in {"q", "Q"}:
        print('AI: 再见，人类')
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream({"input": human, "chat_history": chat_history}):
        # 代理动作
        if "actions" in chunk:
            for action in chunk["actions"]:
                print(f"调用工具 ```{action['tool']}```，输入 ```{action['tool_input']}```")
        # 观察结果
        elif "steps" in chunk:
            for step in chunk["steps"]:
                print(f"得到结果: ```{step['observation']}```")
        # 最终结果
        elif "output" in chunk:
            print(chunk['output'])
            ai = AIMessage(content=chunk['output'])
        else:
            raise ValueError
        print("------")        
    chat_history.extend([HumanMessage(content=human), ai])

AI: 
你好！我是一个非常强大的助手，我可以帮助你解决各种问题和需求。
------
AI: 
是的，我可以搜索网络来获取信息和回答你的问题。
------
AI: 
调用工具 ```search```，输入 ```{'query': '瓲珉'}```
------
得到结果: ```search is not a valid tool, try one of [word_length].```
------
It seems like I can't directly search the internet for information. However, I can help you with other tasks. Is there anything else you need assistance with?
------
AI: 
你welcome！如果你需要更多帮助，请随时问我。
------
AI: 再见，人类


## Stream Events

In [7]:
chat_history = []

while True:
    human = input("人类（输入Q/q退出）: ")
    if human in {"q", "Q"}:
        print('AI: 再见，人类')
        break
    ai = None
    print("AI: ")
    async for event in remote_runnable.astream_events(
        {"input": human, "chat_history": chat_history},
        version="v1",
    ):
        kind = event["event"]
        if kind == "on_chain_start":
            if (
                event["name"] == "agent"
            ):  # 在创建代理时通过 `.with_config({"run_name": "Agent"})` 指定
                print(
                    f"启动代理: {event['name']}，输入: {event['data'].get('input')}"
                )
        elif kind == "on_chain_end":
            if (
                event["name"] == "agent"
            ):  # 在创建代理时通过 `.with_config({"run_name": "Agent"})` 指定
                print()
                print("--")
                print(
                    f"完成代理: {event['name']}，输出: {event['data'].get('output')['output']}"
                )
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # 在 OpenAI 的上下文中，空内容意味着
                # 模型正在请求调用一个工具。
                # 因此我们只打印非空内容
                print(content, end="|")
        elif kind == "on_tool_start":
            print("--")
            print(
                f"启动工具: {event['name']}，输入: {event['data'].get('input')}"
            )
        elif kind == "on_tool_end":
            print(f"完成工具: {event['name']}")
            print(f"工具输出为: {event['data'].get('output')}")
            print("--")

AI: 
启动代理: agent，输入: {'input': '你好', 'chat_history': []}
你|好|！|有什么|可以|帮助|你|？|
--
完成代理: agent，输出: 你好！有什么可以帮助你？
AI: 
启动代理: agent，输入: {'input': '你能帮我什么？', 'chat_history': []}
Hi| there|!| I| can| assist| you| with| a| variety| of| tasks|.| What| do| you| need| help| with| today|?|
--
完成代理: agent，输出: Hi there! I can assist you with a variety of tasks. What do you need help with today?
AI: 
启动代理: agent，输入: {'input': '我希望你一直说中文，请告诉我你能做什么？', 'chat_history': []}
Hi|,| I| can| help| you| with| a| variety| of| tasks|.| I| can| provide| information| on| different| topics|,| help| you| understand| different| languages|,| and| even| assist| you| in| finding| the| length| of| words|.| Just| let| me| know| what| you| need| help| with|!|
--
完成代理: agent，输出: Hi, I can help you with a variety of tasks. I can provide information on different topics, help you understand different languages, and even assist you in finding the length of words. Just let me know what you need help with!
AI: 
启动代理: agent，输入: {'